In [46]:
import json
from os.path import join, dirname
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import pyaudio
import wave

In [55]:
authenticator = IAMAuthenticator('yTSSJ5GSmGhgIA95KnVPDf61KSZinztq909UBMfoqh7l')

In [56]:
speech_to_text = SpeechToTextV1(authenticator=authenticator)

In [57]:
speech_to_text.set_service_url("https://api.us-east.speech-to-text.watson.cloud.ibm.com/instances/77c94867-643f-431b-a593-0bc775c18bb7")

In [58]:
chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 2
fs = 44100  # Record at 44100 samples per second
seconds = 3
filename = "output.wav"

In [59]:
class MyRecognizeCallback(RecognizeCallback):
    def __init__(self):
        RecognizeCallback.__init__(self)

    def on_data(self, data):
        print(json.dumps(data, indent=2))

    def on_error(self, error):
        print('Error received: {}'.format(error))

    def on_inactivity_timeout(self, error):
        print('Inactivity timeout: {}'.format(error))


myRecognizeCallback = MyRecognizeCallback()

In [60]:
p = pyaudio.PyAudio()  # Create an interface to PortAudio
print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording
Finished recording


In [61]:
audio_file = open('output.wav','rb')
audio_source = AudioSource(audio_file)

In [62]:

result = speech_to_text.recognize_using_websocket(
                            audio=audio_source,
                            content_type='audio/wav',
                            recognize_callback=myRecognizeCallback,
                            model='en-US_BroadbandModel',
                            keywords=['left', 'right', 'up','down'],
                            keywords_threshold=0.25,
                            max_alternatives=1)
print(result)
print(type(result))

{
  "results": [
    {
      "keywords_result": {},
      "alternatives": [
        {
          "confidence": 0.99,
          "transcript": "okay "
        }
      ],
      "final": true
    }
  ],
  "result_index": 0
}
None
<class 'NoneType'>


In [45]:
import requests